In [1]:
import pandas as pd

In [2]:
from scrape import aemo, cen

In [3]:
years = range(2016, 2020)

## Australia Energy Consumption

In [4]:
regions = ["NSW", "QLD", "VIC", "SA", "TAS"]

In [5]:
au = aemo.AEMO(regions, years)
au_data = au.get_aemo_data()

In [6]:
len(au.errors)

0

In [7]:
au_data.head()

,REGION,SETTLEMENTDATE,TOTALDEMAND,RRP,PERIODTYPE
0,NSW1,2016-01-01 00:30:00,6976.20,36.82,TRADE
1,NSW1,2016-01-01 01:00:00,6751.26,39.67,TRADE
2,NSW1,2016-01-01 01:30:00,6421.49,38.41,TRADE
3,NSW1,2016-01-01 02:00:00,6125.98,39.41,TRADE
4,NSW1,2016-01-01 02:30:00,5939.91,44.34,TRADE


## Chile Energy Consumption

In [8]:
cl = cen.CEN(years)

In [9]:
cl_data = cl.get_cen_data()

2018


  0%|                                                                                                        | 0/99 [00:00<?, ?it/s]

download_data: 10.49 seconds
unzip_files: 0.05 seconds


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:13<00:00,  7.52it/s]


2017


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 365/365 [03:42<00:00,  1.64it/s]


2016


  0%|                                                                                                       | 0/366 [00:00<?, ?it/s]

download_data: 1.11 minutes
unzip_files: 0.01 seconds


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 366/366 [03:41<00:00,  1.65it/s]


In [10]:
cl_data.head()

,date,Centrales,1,2,3,4,5.0,6,7.0,8.0,...,20,21.0,22.0,23,24.0,Real,Máxima,Prog.,25.0,Total
0,2018-01-01,Total Generación SIC,5586.13,5586.54,5403.36,5168.44,4934.463841,4802.12,4661.904772,4523.939107,...,5411.9,5574.650418,6089.568176,6210.84,5923.152669,127087,6210.84,NaN,NaN,NaN
1,2018-01-02,Total Generación SIC,5563.29,5285.02,5060.24,4951.97,4946.521774,5114.27,5288.953742,5498.294224,...,7101.17,7122.330039,7471.706758,7335.39,6908.765731,157856,7793.84,NaN,NaN,NaN
2,2018-01-03,Total Generación SIC,6467.02,6056.39,5820.87,5669.2,5514.909245,5627.32,5820.001857,5956.964723,...,7039.02,6980.342747,7439.094768,7446.86,6974.695463,166648,7977.25,NaN,NaN,NaN
3,2018-01-04,Total Generación SIC,6503.89,6088.1,5866.77,5723.33,5643.904601,5732.1,5819.773981,6052.061607,...,7087.86,7110.611922,7529.913171,7460.52,7041.212406,165299,7725.31,NaN,NaN,NaN
4,2018-01-05,Total Generación SIC,6506.3,6123.73,5866.21,5730.75,5719.525878,5789.1,5889.826648,6119.921577,...,7062.76,6995.541304,7380.231951,7319.5,7068.237571,165646,7754.07,NaN,NaN,NaN
